In [1]:
import numpy as np
from tqdm import tqdm
import torch
import torch.nn as nn
from torchsummary import summary
import torchvision.transforms as transforms
import json
import random
from pathlib import Path

import argparse

from src.utils.configer import Configer
from torch.utils.data import DataLoader
from src.datasets.TinyImageNetDataset import TinyImageNetDataset
from src.models.customResNet import customResNet
from src.models.customUNet import customUNet
from src.models.customResNetUNet import customResNetUNet

In [2]:
if torch.cuda.is_available():
    print("CUDA is available!")
    print(f"Number of GPUs: {torch.cuda.device_count()}")
    print(f"Current GPU Name: {torch.cuda.get_device_name(0)}")
    print(f"CUDA Version used by PyTorch: {torch.version.cuda}")
else:
    print("CUDA is not available. PyTorch is using CPU.")

CUDA is available!
Number of GPUs: 1
Current GPU Name: NVIDIA GeForce RTX 3060
CUDA Version used by PyTorch: 12.6


In [3]:
def set_seed(seed: int) -> None:
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(seed)
        torch.backends.cudnn.deterministic = True  # To have ~deterministic results
        torch.backends.cudnn.benchmark = False

In [4]:
parser = argparse.ArgumentParser()

parser.add_argument('--disable-cuda', action='store_true',
                    help='Disable CUDA')
parser.add_argument('--hypes', default=None, type=str,
                    dest='hypes', help='The file of the hyper parameters.')
parser.add_argument('--phase', default='train', type=str,
                    dest='phase', help='The phase of module.')
parser.add_argument('--gpu', default=[0, ], nargs='+', type=int,
                    dest='gpu', help='The gpu used.')
parser.add_argument('--resume', default=None, type=str,
                    dest='resume', help='The path of pretrained model.')

args, unknown = parser.parse_known_args()
args.hypes = 'src/hyperparameters/customResNet-config.json'

torch.autograd.set_detect_anomaly(True)
configer = Configer(args)

# Read config-files.
hyperparameters_dir = Path("./src/hyperparameters/")

config_path = hyperparameters_dir / "config.json"
assert config_path.exists(), f"Config not found: {config_path}"
with open(config_path, "r") as f:
    configer.general_config = json.load(f)

set_seed(configer.general_config.get("seed"))

model_config_path = hyperparameters_dir / f"{configer.get('model_name')}-config.json"
assert model_config_path.exists(), f"Config not found: {model_config_path}"
with open(model_config_path, "r") as f:
    configer.model_config = json.load(f)

dataset_config_path = hyperparameters_dir / f"{configer.get('dataset_name')}-config.json"
assert dataset_config_path.exists(), f"Config not found: {dataset_config_path}"
with open(dataset_config_path, "r") as f:
    configer.dataset_config = json.load(f)
    
configer.device = torch.device(configer.general_config.get("device").lower() if torch.cuda.is_available() else 'cpu')
data_path = Path(configer.general_config.get("data_dir")) / configer.get("dataset_name")

In [10]:
checkpoints_file = Path('./checkpoints/customResNet/pretrained_customResNet_4x2_classes_10.pth')

selected_classes = configer.dataset_config.get("selected_classes")
n_classes = len(selected_classes)

mdl_input_size = configer.model_config.get('input_size')

if checkpoints_file is None:
        pretrained_flag = False
else:
        pretrained_flag = True

model, _, _ = customResNet(
        layers_config = configer.model_config.get("layers_num")*[configer.model_config.get("block_size")],
        in_channels = mdl_input_size[0],
        layer0_channels = configer.model_config.get("output_channels") // 2**(configer.model_config.get("layers_num") - 1),
        num_classes = n_classes,
        pretrained = pretrained_flag,
        checkpoints_file = checkpoints_file,
        device = configer.device
        )
model = model.to(configer.device)
model.eval()

test_input = torch.randn(1, *mdl_input_size)
print(configer.model_config.get("layers_num")*[configer.model_config.get("block_size")])
model_size = sum(p.numel() for p in model.parameters() if p.requires_grad)
print(f"Model size: {model_size}")

summary(model, tuple(mdl_input_size))


Restoring checkpoint:  checkpoints\customResNet\pretrained_customResNet_4x2_classes_10.pth
[2, 2, 2, 2]
Model size: 889732
----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 18, 64, 64]           2,646
       BatchNorm2d-2           [-1, 18, 64, 64]              36
              ReLU-3           [-1, 18, 64, 64]               0
         MaxPool2d-4           [-1, 18, 64, 64]               0
            Conv2d-5           [-1, 18, 32, 32]             324
       BatchNorm2d-6           [-1, 18, 32, 32]              36
            Conv2d-7           [-1, 18, 32, 32]           2,916
       BatchNorm2d-8           [-1, 18, 32, 32]              36
              ReLU-9           [-1, 18, 32, 32]               0
           Conv2d-10           [-1, 18, 32, 32]           2,916
      BatchNorm2d-11           [-1, 18, 32, 32]              36
             ReLU-12           [-1, 18, 32, 

In [11]:
mean_norm = [0.485, 0.456, 0.406]
std_norm = [0.229, 0.224, 0.225]

val_transforms = transforms.Compose([
    transforms.Resize(tuple(mdl_input_size[1:])),
    transforms.ToTensor(),
    transforms.Normalize(mean=mean_norm, std=std_norm)
])
val_loader = DataLoader(
    TinyImageNetDataset(
        data_path = data_path,
        split = "val",
        transform = val_transforms,
        selected_classes = selected_classes
        ),
    batch_size=configer.model_config.get("batch_size"),
    shuffle=False,
    num_workers=configer.model_config.get("workers"),
    pin_memory=True)
print(f"Valid. size: {len(val_loader.dataset)}")

Valid. size: 500


In [12]:
loss_func = nn.CrossEntropyLoss().to(configer.device)

total_loss = 0.0
total_correct = 0
total_samples = 0

with torch.no_grad():
    for data_tuple in tqdm(val_loader, desc="Val"):

        inputs, gt = data_tuple[0].to(configer.device), data_tuple[1].to(configer.device)

        output = model(inputs)
        
        loss = loss_func(output, gt)
        
        predicted = torch.argmax(output.detach(), dim=1)

        total_loss += loss.item() * inputs.size(0)
        total_correct += (predicted == gt.detach()).sum().item()
        total_samples += inputs.size(0)

avg_loss = total_loss / total_samples
avg_accuracy = total_correct / total_samples
print(f"Valid. loss: {avg_loss:.4f}, accuracy: {avg_accuracy:.4f}")

Val: 100%|██████████| 1/1 [00:03<00:00,  3.05s/it]

Valid. loss: 1.0363, accuracy: 0.6440


# U-Net

In [13]:
model = customUNet(
    in_channels=3,
    out_channels=1,
    features=[18, 36, 72, 144]
    )
model = model.to(configer.device)

test_input = torch.randn(1, 3, 128, 128).to(configer.device)
test_output = model(test_input)

print(f"   Вход:  {test_input.shape}")
print(f"   Выход: {test_output.shape}")

Encoder features by level: [18, 36, 72, 144]
   Вход:  torch.Size([1, 3, 128, 128])
   Выход: torch.Size([1, 1, 128, 128])


In [14]:
model = customResNetUNet(
    in_channels=3,
    out_channels=1,
    features=[18, 36, 72, 144],
    backbone_layers_config = 4*[2],
    backbone_layer0_channels = 18,
    backbone_pretrained = False,
    backbone_checkpoints_path = "_",
    device = configer.device
    )
model = model.to(configer.device)

test_input = torch.randn(1, 3, 128, 128).to(configer.device)
test_output = model(test_input)

print(f"   Вход:  {test_input.shape}")
print(f"   Выход: {test_output.shape}")

TypeError: customResNet() got an unexpected keyword argument 'checkpoints_path'. Did you mean 'checkpoints_file'?